In [5]:
import os
import json
import re
from datetime import datetime
from typing import TypedDict, Annotated, Sequence, List, Dict, Literal
from operator import add

In [6]:
# LangChain imports
from langchain_community.llms import Ollama
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.tools import tool

In [7]:
# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [8]:
# Pydantic for structured outputs
from pydantic import BaseModel, Field

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
from email.mime.text import MIMEText

# Telegram
import requests

In [9]:
# ============================================================================
# CONFIGURATION
# ============================================================================

class Config:
    # Ollama Settings (Local LLM - Free!)
    OLLAMA_MODEL = "llama3.2:3b"  # or "mistral", "phi3", etc.
    OLLAMA_BASE_URL = "http://localhost:11434"
    
    # Gmail API
    GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
    GMAIL_CREDENTIALS_FILE = 'credentials.json'
    GMAIL_TOKEN_FILE = 'token.json'
    
    # Telegram Bot
    TELEGRAM_BOT_TOKEN = os.environ.get("TELEGRAM_BOT_TOKEN", "")
    TELEGRAM_CHAT_ID = os.environ.get("TELEGRAM_CHAT_ID", "")
    
    # Job Email Categories
    JOB_CATEGORIES = {
        "application_confirmed": {
            "label": "Applied ✓",
            "priority": 2,
            "alert": False,
            "description": "Application received confirmation"
        },
        "interview_request": {
            "label": "Interview 📅",
            "priority": 5,
            "alert": True,
            "description": "Interview invitation or scheduling"
        },
        "interview_reminder": {
            "label": "Interview Reminder ⏰",
            "priority": 4,
            "alert": True,
            "description": "Upcoming interview reminder"
        },
        "offer": {
            "label": "Job Offer 🎉",
            "priority": 5,
            "alert": True,
            "description": "Job offer received"
        },
        "rejected": {
            "label": "Rejected ❌",
            "priority": 3,
            "alert": True,
            "description": "Application rejected"
        },
        "assessment": {
            "label": "Assessment 📝",
            "priority": 4,
            "alert": True,
            "description": "Coding challenge or assessment"
        },
        "follow_up": {
            "label": "Follow-up 💬",
            "priority": 3,
            "alert": False,
            "description": "Follow-up or status update"
        },
        "job_alert": {
            "label": "Job Alert 🔔",
            "priority": 1,
            "alert": False,
            "description": "Job posting from job boards"
        },
        "newsletter": {
            "label": "Newsletter 📰",
            "priority": 0,
            "alert": False,
            "description": "Career newsletters and tips"
        },
        "spam": {
            "label": "Spam 🗑️",
            "priority": 0,
            "alert": False,
            "description": "Spam or promotional content"
        },
        "uncategorized": {
            "label": "Other 📧",
            "priority": 1,
            "alert": False,
            "description": "Cannot be categorized"
        }
    }

In [10]:
# ============================================================================
# DATA MODELS
# ============================================================================

class EmailAnalysis(BaseModel):
    """Structured output for email analysis"""
    category: Literal[
        "application_confirmed",
        "interview_request", 
        "interview_reminder",
        "offer",
        "rejected",
        "assessment",
        "follow_up",
        "job_alert",
        "newsletter",
        "spam",
        "uncategorized"
    ] = Field(description="Email category")
    
    confidence: float = Field(description="Confidence score 0-1", ge=0, le=1)
    reasoning: str = Field(description="Why this category was chosen")
    company_name: str = Field(description="Company name if detected")
    position: str = Field(description="Job position if mentioned")
    action_items: List[str] = Field(description="Action items from the email")
    deadline: str = Field(description="Any deadline mentioned (or 'none')")
    key_info: List[str] = Field(description="Key information extracted")


class EmailState(TypedDict):
    """State for the email processing graph"""
    email_data: Dict
    analysis: Dict
    should_alert: bool
    gmail_label: str
    alert_sent: bool
    actions_taken: List[str]
    error: str

In [11]:
# ============================================================================
# GMAIL INTEGRATION
# ============================================================================

class GmailHandler:
    """Handle Gmail API operations"""
    
    def __init__(self):
        self.service = None
        self.authenticate()
    
    def authenticate(self):
        """Authenticate with Gmail API"""
        creds = None
        
        # Token file stores access and refresh tokens
        if os.path.exists(Config.GMAIL_TOKEN_FILE):
            creds = Credentials.from_authorized_user_file(
                Config.GMAIL_TOKEN_FILE, 
                Config.GMAIL_SCOPES
            )
        
        # If no valid credentials, let user log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    Config.GMAIL_CREDENTIALS_FILE,
                    Config.GMAIL_SCOPES
                )
                creds = flow.run_local_server(port=0)
            
            # Save credentials for next run
            with open(Config.GMAIL_TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())
        
        self.service = build('gmail', 'v1', credentials=creds)
    
    def get_unread_emails(self, max_results: int = 10) -> List[Dict]:
        """Fetch unread emails"""
        try:
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q='is:unread',
                maxResults=max_results
            ).execute()
            
            messages = results.get('messages', [])
            emails = []
            
            for msg in messages:
                email_data = self.get_email_details(msg['id'])
                if email_data:
                    emails.append(email_data)
            
            return emails
            
        except Exception as e:
            print(f"Error fetching emails: {e}")
            return []
    
    def get_email_details(self, msg_id: str) -> Dict:
        """Get full email details"""
        try:
            message = self.service.users().messages().get(
                userId='me',
                id=msg_id,
                format='full'
            ).execute()
            
            headers = message['payload']['headers']
            
            # Extract headers
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '')
            date = next((h['value'] for h in headers if h['name'] == 'Date'), '')
            
            # Extract body
            body = self._get_email_body(message['payload'])
            
            return {
                'id': msg_id,
                'subject': subject,
                'from': sender,
                'date': date,
                'body': body[:2000],  # Limit body size
                'snippet': message.get('snippet', '')
            }
            
        except Exception as e:
            print(f"Error getting email details: {e}")
            return None
    
    def _get_email_body(self, payload):
        """Extract email body from payload"""
        if 'parts' in payload:
            for part in payload['parts']:
                if part['mimeType'] == 'text/plain':
                    if 'data' in part['body']:
                        return base64.urlsafe_b64decode(
                            part['body']['data']
                        ).decode('utf-8')
        elif 'body' in payload and 'data' in payload['body']:
            return base64.urlsafe_b64decode(
                payload['body']['data']
            ).decode('utf-8')
        return ""
    
    def apply_label(self, msg_id: str, label_name: str) -> bool:
        """Apply label to email"""
        try:
            # Get or create label
            label_id = self._get_or_create_label(label_name)
            
            # Apply label and mark as read
            self.service.users().messages().modify(
                userId='me',
                id=msg_id,
                body={
                    'addLabelIds': [label_id],
                    'removeLabelIds': ['UNREAD']
                }
            ).execute()
            
            return True
            
        except Exception as e:
            print(f"Error applying label: {e}")
            return False
    
    def _get_or_create_label(self, label_name: str) -> str:
        """Get existing label or create new one"""
        try:
            # List existing labels
            results = self.service.users().labels().list(userId='me').execute()
            labels = results.get('labels', [])
            
            # Check if label exists
            for label in labels:
                if label['name'] == label_name:
                    return label['id']
            
            # Create new label
            label_object = {
                'name': label_name,
                'labelListVisibility': 'labelShow',
                'messageListVisibility': 'show'
            }
            
            created_label = self.service.users().labels().create(
                userId='me',
                body=label_object
            ).execute()
            
            return created_label['id']
            
        except Exception as e:
            print(f"Error with label: {e}")
            return None

In [4]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import base64

In [5]:

# Gmail API Configuration
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CREDENTIALS_FILE = '../credentials/credentials.json'
TOKEN_FILE = 'token.json'

In [6]:

class GmailHandler:
    """Minimal Gmail fetcher"""

    def __init__(self):
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate with Gmail API"""
        creds = None

        # Load existing token
        if os.path.exists(TOKEN_FILE):
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

        # If no valid token, log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
            # Save token
            with open(TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)

    def get_labels(self):
        """Fetch and display Gmail labels"""
        try:
            results = self.service.users().labels().list(userId='me').execute()
            labels = results.get('labels', [])
            if not labels:
                print("⚠️ No labels found.")
                return []

            print("🏷️  Your Gmail Labels:")
            for label in labels:
                print(f" - {label['name']}")
            print()
            return labels

        except Exception as e:
            print(f"❌ Error fetching labels: {e}")
            return []

    def get_unread_emails(self, max_results=5):
        """Fetch unread emails"""
        try:
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q='is:unread',
                maxResults=max_results
            ).execute()

            messages = results.get('messages', [])
            if not messages:
                print("✅ No unread emails found.")
                return []

            print(f"📬 Found {len(messages)} unread emails:\n")
            emails = []
            for msg in messages:
                details = self.get_email_details(msg['id'])
                if details:
                    emails.append(details)
            return emails

        except Exception as e:
            print(f"❌ Error fetching emails: {e}")
            return []

    def get_email_details(self, msg_id):
        """Get full email details + labels"""
        try:
            msg = self.service.users().messages().get(
                userId='me', id=msg_id, format='full'
            ).execute()

            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '(Unknown)')
            snippet = msg.get('snippet', '')
            labels = msg.get('labelIds', [])

            print(f"📧 From: {sender}")
            print(f"   Subject: {subject}")
            print(f"   Labels: {', '.join(labels) if labels else '(No Labels)'}")
            print(f"   Snippet: {snippet[:80]}...\n")

            return {"id": msg_id, "from": sender, "subject": subject, "labels": labels, "snippet": snippet}

        except Exception as e:
            print(f"❌ Error reading email: {e}")
            return None


if __name__ == "__main__":
    gmail = GmailHandler()

    # 1️⃣ Print all available labels
    gmail.get_labels()

    # 2️⃣ Fetch and print unread emails (with labels)
    emails = gmail.get_unread_emails(max_results=5)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=643813721017-2err0bl1c4ueihcvsgbqm7cckvulsip6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65014%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=kSnMvyZ7yiRCXW466CLaF2Gke7c1p3&access_type=offline
🏷️  Your Gmail Labels:
 - CHAT
 - SENT
 - INBOX
 - IMPORTANT
 - TRASH
 - DRAFT
 - SPAM
 - CATEGORY_FORUMS
 - CATEGORY_UPDATES
 - CATEGORY_PERSONAL
 - CATEGORY_PROMOTIONS
 - CATEGORY_SOCIAL
 - YELLOW_STAR
 - STARRED
 - UNREAD
 - Ghosted
 - Rejection
 - Interview
 - applied1
 - NoUpdates

📬 Found 5 unread emails:

📧 From: Alcumus <alcumus@pinpoint.email>
   Subject: Thanks for Your Application!
   Labels: UNREAD, IMPORTANT, CATEGORY_UPDATES, INBOX
   Snippet: Thanks for Your Application! Hi Abhishek, Thanks for applying for the AI Enginee...

📧 From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
   Subject: “software engineer